In [82]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

In [170]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [84]:
import gc

from sklearn.model_selection import KFold
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error

import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Bidirectional, LSTM
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Concatenate, Add, GRU

np.random.seed(42)
tf.random.set_seed(42)

In [171]:
df = pd.read_csv('/kaggle/input/qqqqqqqq/Traffic_Data.csv')
print(f"train_df: {df.shape}")
df.head()
dft = pd.read_csv('/kaggle/input/aaaaaax/Traffic_Data_Test.csv')
print(f"train_df: {dft.shape}")


In [86]:
df.columns

In [172]:
df['time'] = range(1, len(df) + 1)
dft['time'] = range(1, len(df) + 1)



In [88]:
df.head()

In [109]:
def Time_Series_Features(df):
    df['cross1']= df['car'] * df['truck']
    df['cross2']= df['car'] * df['bus']
    df['cross3']= df['truck'] * df['bus']
   
    
    print("Step-1...Completed")
    
    df['car_lag1'] = df['car'].shift(1)
    df['car_lag2'] = df['car'].shift(-1)
    df['truck_lag1'] = df['truck'].shift(1)
    df['truck_lag2'] = df['truck'].shift(-1)
    
    df['bus_lag1'] = df['bus'].shift(-1)
    df['bus_lag2'] = df['bus'].shift(1)
    
    df = df.fillna(0)
    print("Step-2...Completed")
    
    df['car__max'] = df['car'].max()
    df['truck__max'] = df['truck'].max()
    df['truck__diffmax'] = df['truck'].max() - df['truck']
    df['car__diffmax'] = df['car'].max() - df['car']
    
    df['truck__diffmean'] = df['truck'].mean() - df['truck']
    df['car__diffmean'] = df['car'].mean() - df['car']
    print("Step-3...Completed")
    
    df['car_diff1'] = df['car'] - df['car_lag1']
    df['car_diff2'] = df['car'] - df['car_lag2']
    df['truck_diff1'] = df['truck'] - df['truck_lag1']
    df['trukc_diff2'] = df['truck'] - df['truck_lag2']
    
    print("Step-4...Completed")
    
    
    
    return df

In [173]:
train=Time_Series_Features(df)
tes=Time_Series_Features(dft)

In [114]:
df.plot(x="time",y=["car","truck","bus"],kind='line')
plt.grid()
plt.title("Vehicle no wrt time")
plt.legend()


plt.savefig("Vehicle no wrt time.png",dpi=600)

In [115]:
df.plot(x="time",y=["cross1","cross2","cross3"],kind='line')
plt.grid()
plt.title("Cross of features wrt time")

plt.legend()


plt.savefig("Cross of features wrt time.png",dpi=600)
    

In [116]:
df.plot(x="time",y=["car_lag1","bus_lag1","truck_lag1"],kind='line')
plt.grid()
plt.title("Lag of features wrt time")
plt.legend()


plt.savefig("Lag of features wrt time.png",dpi=600)


In [117]:
df.plot(x="time",y=["car_lag2","bus_lag2","truck_lag2"],kind='line')
plt.grid()
plt.title("Back Lag of features wrt time")
plt.legend()


plt.savefig("Back Lag of features wrt time.png",dpi=600)

In [118]:
train.plot(x="time",y=["truck__diffmax","truck__diffmean","car__diffmax","car__diffmean"],kind='line')
plt.grid()
plt.title("Difference from mean and max wrt time")
plt.legend()


plt.savefig("Difference from mean and max wrt time.png",dpi=600)

In [120]:
df.columns

In [122]:
df.describe()

In [123]:
train.columns

In [174]:
train=train.drop(['frame','time'],axis=1)
tes=tes.drop(['frame','time'],axis=1)



In [175]:
T=np.array(train)
Ttes=np.array(tes)
T.shape

In [126]:
nput=[]
Target=[]

In [127]:
for i in range(1650):
    nput.append(T[0+i:30+i,:])
    Target.append(T[30+i:60+i,:])

In [128]:
nput=np.array(nput)

Target=np.array(Target)
Target.shape,nput.shape

In [129]:
K=nput.shape
print(K[1:])

In [130]:
z=32

In [135]:
def get_model():
    enc_in = Input(shape=(K[1:]))
    x1 = Bidirectional(LSTM(units=128, return_sequences=True))(enc_in)
    x2 = Bidirectional(LSTM(units=64, return_sequences=True))(x1)
    x3 = Bidirectional(LSTM(units=32, return_sequences=True))(x2)
    z2 = Bidirectional(GRU(units=128, return_sequences=True))(x2)
    Ex = Concatenate(axis=2)([x3, z2])
    
    EX = Dense(units=32, activation='selu')(Ex)
    
    

    encoder = tf.keras.Model(enc_in, EX)

    dec_in = Input(shape = (z,1))
    x6 = Bidirectional(LSTM(units=128, return_sequences=True))(EX)
    x7 = Bidirectional(LSTM(units=64, return_sequences=True))(x6)
    x8 = Bidirectional(LSTM(units=32, return_sequences=True))(x7)
    z6 = Bidirectional(GRU(units=128, return_sequences=True))(x8)
    Dx = Concatenate(axis=2)([x8, z6])
    
    DX = Dense(units=32, activation='selu')(Dx)

    dec_out=Dense(units=25)(DX)
    #decoder = tf.keras.Model(dec_in, dec_out)
   

    #dec_out = decoder(encoder(enc_in)[2])

    ae = tf.keras.Model(enc_in, dec_out)

    return {'inputs': enc_in, 
            'outputs': dec_out,
            'encoder': encoder,
            #'decoder': decoder,
            'ae': ae}

In [149]:
model = get_model()
ae=model['ae']
ae.summary()

In [150]:
plot_model(
    ae, 
    to_file='ae.png', 
    show_shapes=True,
    show_layer_names=True
)

In [151]:
ae.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])
history=ae.fit(
    x=nput, y=Target, epochs=500,batch_size=100, validation_split=0.3
)

In [152]:
import matplotlib.pyplot as plt

plt.plot(history.history["val_loss"],label='validation_loss')

plt.plot(history.history["loss"],label='Training_loss')
plt.xlabel("Epochs")
plt.ylabel("Losses")
plt.legend()
plt.grid()
plt.title("Training and Validation Loss")
plt.savefig("Train_valid_loss_mse.png",dpi=600)

plt.show()

In [147]:
ae.save('ae1')

In [169]:
plt.plot(history.history["val_loss"])
plt.xlabel("Epochs")
plt.ylabel("Valid Losses")
plt.legend()
plt.savefig("valid_loss_mse.png",dpi=600)
plt.grid()
plt.show()
plt.plot(history.history["loss"])
plt.xlabel("Epochs")
plt.ylabel("Train Losses")
plt.legend()
plt.grid()
plt.savefig("Train_loss_mse.png",dpi=600)
plt.show()

In [176]:

testin=[]
testarget=[]
for i in range(1650):
    testin.append(Ttes[0+i:30+i,:])
    testarget.append(Ttes[30+i:60+i,:])
testin=np.array(testin)
testarget=np.array(testarget)   

In [177]:
loss, acc = ae.evaluate(testin, testarget)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))